In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import csv
from math import ceil
from random import shuffle

count=0
samples = []
with open('./centre_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        count+=1
        samples.append(line)
with open('./side_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        count+=1
        samples.append(line)
with open('./reverse_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        count+=1
        samples.append(line)
with open('./new_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        count+=1
        samples.append(line)
with open('./lastest_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        count+=1
        samples.append(line)
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        count+=1
        line[0]='/home/user/Documents/sd_car_nanodegree/CarND-Behavioral-Cloning-P3/data/'+line[0]
        samples.append(line)


from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = batch_sample[0]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            #yield sklearn.utils.shuffle(X_train, y_train)
            yield (X_train, y_train)

# Set our batch size
batch_size=128

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

ch, row, col = 3, 160, 320  # Trimmed image format

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda


2021-09-12 19:14:42.536751: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

In [3]:
AlexNet = Sequential()
AlexNet.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(row, col, ch),
        output_shape=(row, col, ch)))
AlexNet.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(64))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))


#Output Layer
AlexNet.add(Dense(1))

#Model Summary
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 40, 80, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 80, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 40, 80, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 40, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 40, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 40, 256)       1

2021-09-12 19:14:44.348316: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-12 19:14:44.348810: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-12 19:14:44.395533: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-12 19:14:44.395558: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: user-desktop
2021-09-12 19:14:44.395564: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: user-desktop
2021-09-12 19:14:44.395631: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.63.1
2021-09-12 19:14:44.395650: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.63.1
2021-09-12 19:14:44.395656: I tensorflow/stream_execu

In [4]:
AlexNet.compile(loss='mse', optimizer='adam')

In [5]:
AlexNet.fit_generator(train_generator,
            steps_per_epoch=ceil(len(train_samples)/batch_size),
            validation_data=validation_generator,
            validation_steps=ceil(len(validation_samples)/batch_size),
            epochs=10, verbose=1)

/home/user/anaconda3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-12 19:14:49.172583: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-12 19:14:49.190154: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz


Epoch 1/10
127/127 [==============================] - 174s 1s/step - loss: 0.5508 - val_loss: 0.0216
Epoch 2/10
127/127 [==============================] - 176s 1s/step - loss: 0.0184 - val_loss: 0.0227
Epoch 3/10
127/127 [==============================] - 176s 1s/step - loss: 0.0168 - val_loss: 0.0195
Epoch 4/10
127/127 [==============================] - 176s 1s/step - loss: 0.0172 - val_loss: 0.0202
Epoch 5/10
127/127 [==============================] - 175s 1s/step - loss: 0.0170 - val_loss: 0.0194
Epoch 6/10
127/127 [==============================] - 176s 1s/step - loss: 0.0169 - val_loss: 0.0190
Epoch 7/10
127/127 [==============================] - 176s 1s/step - loss: 0.0158 - val_loss: 0.0195
Epoch 8/10
127/127 [==============================] - 176s 1s/step - loss: 0.0157 - val_loss: 0.0187
Epoch 9/10
127/127 [==============================] - 175s 1s/step - loss: 0.0157 - val_loss: 0.0174
Epoch 10/10
127/127 [==============================] - 176s 1s/step - loss: 0.0155 - val_lo

In [6]:
AlexNet.save('model.h5')

In [7]:
AlexNet.fit_generator(train_generator,
            steps_per_epoch=ceil(len(train_samples)/batch_size),
            validation_data=validation_generator,
            validation_steps=ceil(len(validation_samples)/batch_size),
            epochs=5, verbose=1)

Epoch 1/5
127/127 [==============================] - 174s 1s/step - loss: 0.0157 - val_loss: 0.0182
Epoch 2/5
127/127 [==============================] - 175s 1s/step - loss: 0.0151 - val_loss: 0.0183
Epoch 3/5
127/127 [==============================] - 175s 1s/step - loss: 0.0153 - val_loss: 0.0225
Epoch 4/5
 74/127 [================>.............] - ETA: 1:09 - loss: 0.0144

KeyboardInterrupt: 